# Sample script for calculating cloud adjustments

Closely follows script on Mark Zelinka's Github page, but doesn't use his personal libraries. 

To calculate cloud adjustments the first step is to download Mark's cloud radiative kernels, available [here](https://github.com/mzelinka/cloud-radiative-kernels/tree/master/data). Then for each model you need monthly cloud fractions binned using the ISCCP simulator ("clisccp") and also the surface clear-sky shortwave radiation ("rsuscs" and "rsdscs"), to calculate the surface albedo. The code is set up to loop through multiple models, though only one model is used here.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import scipy.interpolate as si

/home/nick/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/nick/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
#First load cloud kernels, mask nan values
direc = "/home/nlutsko/data/CMIP5/ERF/"

f=xr.open_dataset(direc+'cloud_kernels2.nc')
LWkernel=f.LWkernel
SWkernel=f.SWkernel
klats=f.lat[:]
l = len(klats)
f.close()

#sometimes have nan issues, so I'm pretty cautious about masking everything out
LWkernel = np.array(LWkernel[:,:])
LWkernel = np.ma.masked_where(np.isnan(LWkernel),LWkernel)
SWkernel = np.array(SWkernel[:,:])
SWkernel = np.ma.masked_where(np.isnan(SWkernel),SWkernel)

In [ ]:
#Calculate annual-mean adjustments for aquaplanet simulation with MRI-ESM2-0:
direc2 = "/home/nlutsko/data/CMIP6/Forcing/Aqua_fixed/"

Model = ["MRI-ESM2-0"]

m = len(Mod)

lats = np.linspace( -90., 90., 64 )
lons = np.linspace( 0., 360., 128)

for z in range( m ):
    print("Doing: ", Model[z])

    #Load cloud fractions and take difference
    ds=xr.open_mfdataset(direc2 + "clisccp_CFmon_" + Mod[z] + "_aqua-control*")
    clouds_PI = ds.clisccp
    mlats = np.array(ds.lat[:])
    mlons = ds.lon[:]
    ds.close()
    ds=xr.open_mfdataset(direc2 + "clisccp_CFmon_" + Mod[z] + "_aqua-4xCO2*")
    clouds_4x = ds.clisccp
    ds.close()
    print("Loaded data from files")

    clouds_PI_sc = clouds_PI.groupby('time.month').mean(dim="time")
    clouds_4x_sc = clouds_4x.groupby('time.month').mean(dim="time")
    anom_clouds = clouds_4x_sc - clouds_PI_sc
    print("Taken monthly means")
    print(np.shape(anom_clouds)) #Make sure dimensions look good

    #Regrid to kernel grid:
    #Set NANs to 0 for interpolation
    anom_clouds = np.nan_to_num(anom_clouds)
    if z == 1 or z == 3:
        print ("here")
        mlats[0] = -89.
        mlats[-1] = 89.
    f = si.interp1d( mlats, anom_clouds, axis = 3, kind = 'cubic') 
    regrid_anom_clouds = f( klats) #Interpolate to kernel grid
    print("Regrided cloud anomalies")
    
    LW_cld_adj = LWkernel[:, :, :, :, np.newaxis, 0] *regrid_anom_clouds
    annual_mean_LW_cld_adj = np.mean(LW_cld_adj, axis = 0)
    annual_mean_LW_cld_adj = np.sum(np.sum(annual_mean_LW_cld_adj[:, :1], axis = 0), axis = 0)
    print("Have LW cloud feedback")
    
    #For SW cloud feedback need surface albedo
    #Start by calculating albedo
    ds = xr.open_mfdataset(direc2 + "rsuscs_Amon_" + Mod[z] + "_aqua-control*")
    rsuscs1 = ds.rsuscs[:]
    ds.close()
    ds = xr.open_mfdataset(direc2 + "rsdscs_Amon_" + Mod[z] + "_aqua-control*")
    rsdscs1 = ds.rsdscs[:]
    ds.close()

    albedo=rsuscs1/rsdscs1
    albedo = albedo.groupby('time.month').mean(dim="time")
    del(rsuscs1,rsdscs1)
    print ("Have surface albedo")
    
    albedo = np.array(albedo)
    #Set NANs to 1
    d1, d2, d3 = np.shape(albedo)
    for i in range(d1):
        for j in range(d2):
            albedo[i, j, np.isnan(albedo[i, j, :])] = 1.
            albedo[i, j, np.isinf(albedo[i, j, :])] = 1.

    SWk = SWkernel[:, :, :, :, :]
        
    d1, d2, d3, d4, d5 = np.shape(SWk)
    t, llat, llon = np.shape( albedo)

    print("Interpolate albedos")
    new_albedos = np.zeros( ( (12, d4, llon)))
    f = si.interp1d( mlats, albedo, axis = 1, kind = 'cubic')  
    new_albedo = f( klats) #Interpolate to kernel grid  

    print("Regrid SWkernel")
    #Regrid SWkernel to match albedos. Assume can add kernels linearly:
    nSWkernel = np.zeros( ( ( ( (d1, d2, d3, d4, llon)))))
    for k in range( 12 ):
        print(k)
        for i in range( llon ):
            for h in range( d4 - 2 ):
                if new_albedo[k, h, i] == 0.:
                    nSWkernel[k, :, :, h, i] = SWk[k, :, :, h, 0]
                elif new_albedo[k, h, i] > 0. and new_albedos[k, h, i] < 0.5:
                    c = new_albedo[k, h, i] / 0.5
                    nSWkernel[k, :, :, h, i] = (1. - c) * SWk[k, :, :, h, 0] + c * SWk[k, :, :, h, 1]
                elif new_albedo[k, h, i] == 0.5:
                    nSWkernel[k, :, :, h, i] = SWk[k, :, :, h, 1]
                elif new_albedo[k, h, i] > 0.5 and new_albedo[k, h, j] < 1.0:
                    c = (new_albedo[k, h, i] - 0.5) / 0.5
                    nSWkernel[k, :, :, h, i] = (1. - c) * SWk[k, :, :, h, 1] + c * SWk[k, :, :, h, 2]
                elif new_albedo[k, h, i] >= 1. and new_albedo[k, h, i] < 2.:
                    nSWkernel[k, :, :, h, i] = SWk[k, :, :, h, 2]
                elif new_albedo[k, h, i] > -.2 and new_albedo[k, h, i] < 0.:
                    nSWkernel[k, :, :, h, i] = SWk[k, :, :, h, 0]
                else:
                    print(new_albedo[k, h, i], "Problem")
          
    #Finally can calculate SW feedback
    SW_cld_adj = nSWkernel[:, :, :, :, :] *regrid_anom_clouds
    annual_mean_SW_cld_adj = np.mean(SW_cld_adj, axis = 0)
    annual_mean_SW_cld_adj = np.sum(np.sum(annual_mean_SW_cld_adj[:, :1], axis = 0), axis = 0)
    print("Have annual-mean SW cloud adjustment")
    
    net_cld_adj = annual_mean_LW_cld_adj + annual_mean_SW_cld_adj
    
    #Take global-mean
    rad_lats = klats * np.pi / 180.
    rad_lons = mlons * np.pi / 180.
    cos_lats = np.cos(rad_lats)
    cos_lats = np.expand_dims(cos_lats, axis = 1)
    global_mean_cld_adj = np.trapz(np.trapz(net_cld_adj * cos_lats, rad_lons, axis = 1), rad_lats) / 4. / np.pi
    print(global_mean_cld_adj)
    
    #Interpolate onto common grid
    f = si.interp2d( mlons, klats, net_cld_adj, kind = 'cubic')
    CMIP6_aqua_net_adj[z] = f( lons, lats)

#save
np.save("CMIP6_aqua_cloud_adj", u=CMIP6_aqua_net_adj)
    

### Scott et al adjustment
[Scott et al. (2020)](https://journals.ametsoc.org/view/journals/clim/33/18/jcliD191028.xml) developed a method for calculating low cloud feedbacks which removes the effects of mid- and high-level cloud masking. We adapted it to calculate low cloud adjustments by modifying the Zelinka et al procedure slightly

In [ ]:
for z in range( m ):
    print("Doing: ", Model[z])

    #Load cloud fractions and take difference
    ds=xr.open_mfdataset(direc2 + "clisccp_CFmon_" + Mod[z] + "_aqua-control*")
    clouds_PI = ds.clisccp
    mlats = np.array(ds.lat[:])
    mlons = ds.lon[:]
    ds.close()
    ds=xr.open_mfdataset(direc2 + "clisccp_CFmon_" + Mod[z] + "_aqua-4xCO2*")
    clouds_4x = ds.clisccp
    ds.close()
    print("Loaded data from files")
    
    #Now the various things we need for the Scott et al adjustment
    fbaro = clouds_PI.groupby('time.month').mean(dim="time")
    fnewo = clouds_4x.groupby('time.month').mean(dim="time")
    
    fbaro = np.nan_to_num(fbaro)
    fnewo = np.nan_to_num(fnewo)

    f = si.interp1d( mlats, fbaro, axis = 3, kind = 'cubic') 
    f2 = si.interp1d( mlats, fnewo, axis = 3, kind = 'cubic') 

    nklats = np.array(klats)[1:l - 1]
    fbar = f( nklats) #Interpolate to kernel grid
    fnew = f2( nklats) #Interpolate to kernel grid
    print("Regrided cloud anomalies")  
    
    fdash = fnew - fbar
    
    Lbar = np.sum(np.sum(fbar[:, :, :2], axis = 1), axis = 1)
    Ubar = np.sum(np.sum(fbar[:, :, 2:], axis = 1), axis = 1)
    Lnew = np.sum(np.sum(fnew[:, :, :2], axis = 1), axis = 1)
    Unew = np.sum(np.sum(fnew[:, :, 2:], axis = 1), axis = 1)
    
    Lnbar = Lbar / (1. - Ubar)
    Lnnew = Lnew / (1. - Unew)
    Lndash = Lnnew - Lnbar
    
    Ldash = Lndash * (1. - Ubar)
    
    fdd = fdash - fbar * Ldash[:, np.newaxis, np.newaxis, :, :] / Lbar[:, np.newaxis, np.newaxis, :, :]
    #Need this for the adjustment
    
    #LW cloud feedback is easy, doesn't depend on surface albedo
    low_LWkernel = LWkernel[:, :, :2, 1:l - 1]
    temp1 = fbar[:, :, :2] / Lbar[:, np.newaxis, np.newaxis, :, :] * Ldash[:, np.newaxis, np.newaxis, :, :]
    LW_cld_adj = low_LWkernel[:, :, :, :, np.newaxis, 0] * (temp1 + fdd[:, :, :2])

    annual_mean_LW_cld_adj = np.mean(LW_cld_adj, axis = 0)
    annual_mean_LW_cld_adj = np.sum(np.sum(annual_mean_LW_cld_adj[:, :], axis = 0), axis = 0)
    
    rad_lats = nklats * np.pi / 180.
    rad_lons = mlons * np.pi / 180.
    cos_lats = np.cos(rad_lats)
    cos_lats = np.expand_dims(cos_lats, axis = 1)

    l1 = np.where( abs(nklats) <= 60.)
    
    global_mean_lw_cld_adj = np.trapz(np.trapz(annual_mean_LW_cld_adj[l1, :] * cos_lats[l1], 
                                               rad_lons, axis = 2), rad_lats[l1]) / 4. / np.pi
    print("GM LW adjustment: ", global_mean_lw_cld_adj)    
    
    #For SW cloud feedback need surface albedo
    #Start by calculating albedo
    ds = xr.open_mfdataset("/data/FixedSST_Exps/CMIP5/sstClim/rsuscs/rsuscs_Amon_" + cmip5_mod[z] + "_sstClim_r1i1p1_*")
    rsuscs1 = ds.rsuscs[:]
    ds.close()
    ds = xr.open_mfdataset("/data/FixedSST_Exps/CMIP5/sstClim/rsdscs/rsdscs_Amon_" + cmip5_mod[z] + "_sstClim_r1i1p1_*")
    rsdscs1 = ds.rsdscs[:]
    ds.close()
    
    albedo=rsuscs1/rsdscs1
    albedo = albedo.groupby('time.month').mean(dim="time")
    del(rsuscs1,rsdscs1)
    print ("Have surface albedo")
    
    albedo = np.array(albedo)
    #Set NANs to 1
    d1, d2, d3 = np.shape(albedo)
    for i in range(d1):
        for j in range(d2):
            albedo[i, j, np.isnan(albedo[i, j, :])] = 1.
            albedo[i, j, np.isinf(albedo[i, j, :])] = 1.

    low_SWkernel = SWkernel[:, :, :2, 1:l - 1]
        
    d1, d2, d3, d4, d5 = np.shape(low_SWkernel )
    t, llat, llon = np.shape( albedo)
    
    print("Interpolate albedos")
    new_albedos = np.zeros( ( (12, d4, llon)))
    f = si.interp1d( mlats, albedo, axis = 1, kind = 'cubic')  
    new_albedo = f( nklats) #Interpolate to kernel grid         

    print("Regrid SWkernel")
    #Regrid SWkernel to match albedos. Assume can add kernels linearly:
    nSWkernel = np.zeros( ( ( ( (d1, d2, d3, d4, llon)))))
    for k in range( 12 ):
        print(k)
        for i in range( llon ):
            for h in range( d4 - 2 ):
                if new_albedo[k, h, i] == 0.:
                    nSWkernel[k, :, :, h, i] = low_SWkernel[k, :, :, h, 0]
                elif new_albedo[k, h, i] > 0. and new_albedos[k, h, i] < 0.5:
                    c = new_albedo[k, h, i] / 0.5
                    nSWkernel[k, :, :, h, i] = (1. - c) * low_SWkernel[k, :, :, h, 0] + c * low_SWkernel[k, :, :, h, 1]
                elif new_albedo[k, h, i] == 0.5:
                    nSWkernel[k, :, :, h, i] = low_SWkernel[k, :, :, h, 1]
                elif new_albedo[k, h, i] > 0.5 and new_albedo[k, h, j] < 1.0:
                    c = (new_albedo[k, h, i] - 0.5) / 0.5
                    nSWkernel[k, :, :, h, i] = (1. - c) * low_SWkernel[k, :, :, h, 1] + c * low_SWkernel[k, :, :, h, 2]
                elif new_albedo[k, h, i] >= 1. and new_albedo[k, h, i] < 2.:
                    nSWkernel[k, :, :, h, i] = low_SWkernel[k, :, :, h, 2]
                elif new_albedo[k, h, i] > -.2 and new_albedo[k, h, i] < 0.:
                    nSWkernel[k, :, :, h, i] = low_SWkernel[k, :, :, h, 0]
                else:
                    print(new_albedo[k, h, i], "Problem")
 

    SW_cld_adj = nSWkernel[:, :, :, :, np.newaxis, 0] * (temp1 + fdd[:, :, :2])
    annual_mean_SW_cld_adj = np.mean(SW_cld_adj, axis = 0)
    annual_mean_SW_cld_adj = np.sum(np.sum(annual_mean_SW_cld_adj[:, :], axis = 0), axis = 0)
    print("Have annual-mean SW cloud adjustment")
    global_mean_sw_cld_adj = np.trapz(np.trapz(annual_mean_SW_cld_adj[l1, :] * cos_lats[l1], 
                                               rad_lons, axis = 2), rad_lats[l1]) / 4. / np.pi
    print("GM SW adjustment: ", global_mean_sw_cld_adj)
    
    net_cld_adj = annual_mean_LW_cld_adj + annual_mean_SW_cld_adj
    global_mean_cld_adj = np.trapz(np.trapz(net_cld_adj[l1, :] * cos_lats[l1], 
                                               rad_lons, axis = 2), rad_lats[l1]) / 4. / np.pi
    print("GM adjustment: ", global_mean_cld_adj)
    
    low_net_adj = global_mean_cld_adj
    low_lw_adj = global_mean_lw_cld_adj
    low_sw_adj = global_mean_sw_cld_adj  
